In [ ]:
import tensorflow as tf

tf.random.set_seed(2024)

In [ ]:
import numpy as np

def rgb_to_gray(images):
    images = images.astype(float)
    # Grayscale conversion formula: Y = 0.299R + 0.587G + 0.114B
    grayscale_images = np.dot(images, [0.299, 0.587, 0.114])
    return grayscale_images

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
import numpy as np

def define_paths(dir):
    filepaths = []
    labels = []
    folds = os.listdir(dir)
    for fold in folds:
        foldpath = os.path.join(dir, fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath, file)
            filepaths.append(fpath)
            labels.append(fold)
    return filepaths, labels

def create_dataset(tr_dir, val_dir, ts_dir):
    # train dataframe
    files, labels = define_paths(tr_dir)
    arrays = []
    i=0
    for file in files:
        img = load_img(file, target_size=(100, 100))
        img_array = img_to_array(img)
        arrays.append(img_array)

    arrays = np.array(arrays)
    arrays = rgb_to_gray(arrays)
    arrays = arrays.reshape((len(arrays), 10000)) / 255

    encoder = LabelEncoder()
    labels = np.array(encoder.fit_transform(labels)).reshape((len(labels),))

    unique_values, counts = np.unique(labels, return_counts=True)

    print(arrays.shape, labels.shape)

    X_train = arrays
    y_train = labels


    # valid dataframe
    files, labels = define_paths(val_dir)
    arrays = []
    for file in files:
        img = load_img(file, target_size=(100, 100))
        img_array = img_to_array(img)
        arrays.append(img_array)

    arrays = np.array(arrays)
    arrays = rgb_to_gray(arrays)
    arrays = arrays.reshape((len(arrays), 10000)) / 255

    labels = np.array(encoder.fit_transform(labels)).reshape((len(labels),))

    print(arrays.shape, labels.shape)

    X_valid = arrays
    y_valid = labels

    # test dataframe
    files, labels = define_paths(ts_dir)
    arrays = []
    for file in files:
        img = load_img(file, target_size=(100, 100))
        img_array = img_to_array(img)
        arrays.append(img_array)

    arrays = np.array(arrays)
    arrays = rgb_to_gray(arrays)
    arrays = arrays.reshape((len(arrays), 10000)) / 255

    labels = np.array(encoder.fit_transform(labels)).reshape((len(labels),))

    print(arrays.shape, labels.shape)

    X_test = arrays
    y_test = labels

    return unique_values, counts, X_train, y_train, X_valid, y_valid, X_test, y_test, encoder

In [ ]:
import matplotlib.pyplot as plt

def show_images(X_train, y_train, encoder):
    # Assuming the dataframe has 'image' column for images and 'label' column for labels
    images = X_train
    labels = encoder.inverse_transform(y_train)

    classes = np.unique(labels)

    plt.figure(figsize=(20, 20))
    rows = int(np.ceil(40 / 5))
    for i in range(20):
        plt.subplot(rows, 5, i + 1)
        image = images[i+300]
        plt.imshow(image.reshape(100,100,1))
        plt.axis('off')
    plt.show()

In [ ]:
train_dir = '/kaggle/input/cards-image-datasetclassification/train'
valid_dir = '/kaggle/input/cards-image-datasetclassification/valid'
test_dir = '/kaggle/input/cards-image-datasetclassification/test'
classes, counts, X_train, y_train, X_valid, y_valid, X_test, y_test, encoder = create_dataset(train_dir, valid_dir, test_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/cards-image-datasetclassification/train'

In [ ]:
# Gráfico para visualizar a frequência de cada carta
# Crie uma tabela de frequências
print("Valor   Frequência")
print("------------------")
for value, count in zip(classes, counts):
    print(f"{value:5}   {count:10}")

In [ ]:
# Escrever os valores da tabela anterior em linhas para que seja possível escrever corretamente no relatório (diretamente)

all_labels = encoder.inverse_transform(classes)

len_classes = len(classes)

# Adicionar '&' entre os valores das contagens
for i in range(1, len(all_labels) * 2 - 1, 2):
    all_labels = np.insert(all_labels, i, '&')

In [ ]:
print(all_labels)

In [ ]:
print(counts)

In [ ]:
show_images(X_train, y_train, encoder)

In [ ]:
# Dicionário de hiperparâmetros
hiperparameters = {
    'learning_rate': [0.001, 0.01],
    'neurons': [128, 512],
    'activation': ['relu', 'softmax'],
    'epochs': [15, 30]
}

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import time

def build_and_train_model(learning_rate, neurons, activation, epochs):
    model_MLP = tf.keras.Sequential([
        tf.keras.layers.Dense(neurons, activation=activation, input_shape=(10000,)),
        tf.keras.layers.Dense(53, activation='softmax')
    ])

    # Compile the model
    model_MLP.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    # Train the model
    inicio = time.time()
    history_MLP = model_MLP.fit(X_train, y_train, epochs=epochs, validation_data = (X_valid, y_valid))
    fim = time.time()
    tempo_total = fim - inicio

    test_loss, test_acc = model_MLP.evaluate(X_test, y_test)

    return history_MLP, test_loss, test_acc, model_MLP, tempo_total

In [ ]:
from itertools import product
import math

all_results = []
best_model_MLP = [0,None,0,0]

i = 1
# Iterar sobre todas as combinações de hiperparâmetros
for learning_rate, neurons, activation, epochs in product(*hiperparameters.values()):
    print(f'*** Teste nº:{i} ***')
    print(f"Treinando modelo com learning_rate={learning_rate}, num_neurons={neurons}, funcao_de_ativacao={activation}, numero_de_epochs={epochs}")
    history_MLP, test_loss, test_acc, model_MLP, tempo_total = build_and_train_model(learning_rate, neurons, activation, epochs)
    print(f'Accuracy no conjunto de teste: {test_acc}')
    all_results.append([test_loss, test_acc, learning_rate, neurons, activation, epochs, tempo_total])

    if best_model_MLP[3] < test_acc:
        best_model_MLP = [i, history_MLP, test_loss, test_acc, model_MLP, learning_rate, neurons, activation, epochs, tempo_total]

    i = i+1


In [ ]:
best_model_MLP

In [ ]:
import os
import tensorflow as tf

# Ensure that 'modelos_guardados' directory exists
#os.makedirs('modelos_guardados', exist_ok=True)

# Save the model in the specified directory
model = best_model_MLP[4]
model.save('best_model_CARDS_MLP.h5')

In [ ]:
# Guardar todos os resultados num csv
import csv
import os

csv_file_path = "resultados_CARDS/tuning_results_CARDS_MLP.csv"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

fieldnames = ["model_execution", "epochs", "learning_rate", "neurons", "activation_function", "test_loss", "accuracy", "tempo_execução"]

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    model_execution = 1
    for result in all_results:
        [test_loss, test_acc, learning_rate, neurons, activation, epochs, tempo] = result
        writer.writerow({
            "model_execution": model_execution,
            "epochs": epochs,
            "learning_rate": learning_rate,
            "neurons": neurons,
            "activation_function": activation,
            "test_loss": test_loss,
            "accuracy": test_acc,
            "tempo_execução": tempo
        })
        model_execution += 1

print("CSV file created successfully.")

## Gráficos

In [ ]:
import matplotlib.pyplot as plt

history_MLP = best_model_MLP[1]

# Plotting accuracy
plt.plot(history_MLP.history['accuracy'])
plt.plot(history_MLP.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Treino', 'Validação'], loc='upper left')
plt.show()

In [ ]:
# Gráfico evolução Loss
plt.plot(history_MLP.history['loss'])
plt.plot(history_MLP.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Treino', 'Validação'], loc='upper left')
plt.show()

### Matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

model_MLP = best_model_MLP[4]

y_pred_fc = np.argmax(model_MLP.predict(X_test), axis=1)

# confusion matrix
cm_fc = confusion_matrix(y_true=y_test, y_pred=y_pred_fc)

In [ ]:
import seaborn as sns
import pandas as pd

# Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_fc, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title('Matriz de Confusão')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()